# Dataset Overview

In [1]:
import pandas as pd
df = pd.read_csv('public.csv')
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15565701,Ferri,698,Spain,Female,39,9,161993.89,1,0,0,90212.38,0
1,15565706,Akobundu,612,Spain,Male,35,1,0.00,1,1,1,83256.26,1
2,15565796,Docherty,745,Germany,Male,48,10,96048.55,1,1,0,74510.65,0
3,15565806,Toosey,532,France,Male,38,9,0.00,2,0,0,30583.95,0
4,15565878,Bates,631,Spain,Male,29,3,0.00,2,1,1,197963.46,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,15815628,Moysey,711,France,Female,37,8,113899.92,1,0,0,80215.20,0
7996,15815645,Akhtar,481,France,Male,37,8,152303.66,2,1,1,175082.20,0
7997,15815656,Hopkins,541,Germany,Female,39,9,100116.67,1,1,1,199808.10,1
7998,15815660,Mazzi,758,France,Female,34,1,154139.45,1,1,1,60728.89,0


# Use Pyspark to view dataset 

In [2]:
!pip install findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# These part is  for windows version, if you use ubuntu, remember to edit import pyspark part
# ----
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
# ----
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Churn_Modelling").getOrCreate()
df = spark.read.csv('public.csv',header=True,inferSchema=True)
df.printSchema()

root
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



# Do your work here

In [4]:
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil import parser
from pyspark.sql.functions import unix_timestamp, date_format, col, when
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel
from pyspark.ml.feature import RFormula
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [5]:
cust_df = df.select('CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited')#.filter((df.Age > 40) & (df.Age < 70))

In [6]:
# Because the sample uses an algorithm that works only with numeric features, convert them so they can be consumed
sI1 = StringIndexer(inputCol="Geography", outputCol="GeographyIndex")
en1 = OneHotEncoder(dropLast=False, inputCol="GeographyIndex", outputCol="GeographyVec")
sI2 = StringIndexer(inputCol="Gender", outputCol="GenderIndex")
en2 = OneHotEncoder(dropLast=False, inputCol="GenderIndex", outputCol="GenderVec")

# Create a new DataFrame that has had the encodings applied
encoded_final_df = Pipeline(stages=[sI1, en1, sI2, en2]).fit(cust_df).transform(cust_df)

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, MultilayerPerceptronClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=[
   'CreditScore', 'GeographyVec', 'GenderVec', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'
], outputCol='features')

# Consolidate predictor columns
cust_assembled = assembler.transform(encoded_final_df)

# Check the resulting column
cust_assembled.select('features', 'Exited').show(20, truncate=False)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = cust_assembled.randomSplit([0.7, 0.3], 2)

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="Exited", featuresCol="features", maxDepth=11, maxBins=32)

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[dt])

# Train model.  This also runs the indexers.
dtModel = pipeline.fit(trainingData)

# Make predictions.
predictions = dtModel.transform(testData)

# Select example rows to display.
predictions.select("prediction", "Exited").show(20)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="Exited", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

+--------------------------------------------------------------------+------+
|features                                                            |Exited|
+--------------------------------------------------------------------+------+
|[698.0,0.0,0.0,1.0,0.0,1.0,39.0,9.0,161993.89,1.0,0.0,0.0,90212.38] |0     |
|[612.0,0.0,0.0,1.0,1.0,0.0,35.0,1.0,0.0,1.0,1.0,1.0,83256.26]       |1     |
|[745.0,0.0,1.0,0.0,1.0,0.0,48.0,10.0,96048.55,1.0,1.0,0.0,74510.65] |0     |
|(13,[0,1,4,6,7,9,12],[532.0,1.0,1.0,38.0,9.0,2.0,30583.95])         |0     |
|[631.0,0.0,0.0,1.0,1.0,0.0,29.0,3.0,0.0,2.0,1.0,1.0,197963.46]      |0     |
|[845.0,1.0,0.0,0.0,0.0,1.0,28.0,9.0,0.0,2.0,1.0,1.0,56185.98]       |0     |
|[653.0,1.0,0.0,0.0,1.0,0.0,44.0,8.0,0.0,2.0,1.0,1.0,154639.72]      |0     |
|[497.0,0.0,1.0,0.0,1.0,0.0,41.0,5.0,80542.81,1.0,0.0,0.0,88729.22]  |1     |
|[545.0,0.0,0.0,1.0,0.0,1.0,32.0,4.0,0.0,1.0,1.0,0.0,94739.2]        |0     |
|[596.0,1.0,0.0,0.0,1.0,0.0,39.0,9.0,0.0,1.0,1.0,0.0,48963.59]  

In [8]:
datestamp = datetime.now().strftime('%m-%d-%Y-%s')
fileName = "dtModel_" + datestamp
logRegDirfilename = fileName
dtModel.save(logRegDirfilename)

# Public Evaluation Part

## Load private dataset, the same structure as public dataset

In [9]:
df_private_evl = spark.read.csv('public.csv',header=True,inferSchema=True)  # TA takes public dataset as example

## Do prediction with your PySpark model here (**Very Important**)
You must do prediction (inference) here.
Then TA will know how to run your ML model to test private data.

In [10]:
cust_df_private = df_private_evl.select('CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited')

In [11]:
# Because the sample uses an algorithm that works only with numeric features, convert them so they can be consumed
sI1 = StringIndexer(inputCol="Geography", outputCol="GeographyIndex")
en1 = OneHotEncoder(dropLast=False, inputCol="GeographyIndex", outputCol="GeographyVec")
sI2 = StringIndexer(inputCol="Gender", outputCol="GenderIndex")
en2 = OneHotEncoder(dropLast=False, inputCol="GenderIndex", outputCol="GenderVec")

# Create a new DataFrame that has had the encodings applied
encoded_final_df_private = Pipeline(stages=[sI1, en1, sI2, en2]).fit(cust_df_private).transform(cust_df_private)

In [12]:
dtModel.load(logRegDirfilename)

PipelineModel_1352b00af176

In [13]:
assembler = VectorAssembler(inputCols=[
   'CreditScore', 'GeographyVec', 'GenderVec', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'
], outputCol='features')

# Consolidate predictor columns
cust_assembled = assembler.transform(encoded_final_df_private)

# Check the resulting column
cust_assembled.select('features', 'Exited').show(20, truncate=False)

+--------------------------------------------------------------------+------+
|features                                                            |Exited|
+--------------------------------------------------------------------+------+
|[698.0,0.0,0.0,1.0,0.0,1.0,39.0,9.0,161993.89,1.0,0.0,0.0,90212.38] |0     |
|[612.0,0.0,0.0,1.0,1.0,0.0,35.0,1.0,0.0,1.0,1.0,1.0,83256.26]       |1     |
|[745.0,0.0,1.0,0.0,1.0,0.0,48.0,10.0,96048.55,1.0,1.0,0.0,74510.65] |0     |
|(13,[0,1,4,6,7,9,12],[532.0,1.0,1.0,38.0,9.0,2.0,30583.95])         |0     |
|[631.0,0.0,0.0,1.0,1.0,0.0,29.0,3.0,0.0,2.0,1.0,1.0,197963.46]      |0     |
|[845.0,1.0,0.0,0.0,0.0,1.0,28.0,9.0,0.0,2.0,1.0,1.0,56185.98]       |0     |
|[653.0,1.0,0.0,0.0,1.0,0.0,44.0,8.0,0.0,2.0,1.0,1.0,154639.72]      |0     |
|[497.0,0.0,1.0,0.0,1.0,0.0,41.0,5.0,80542.81,1.0,0.0,0.0,88729.22]  |1     |
|[545.0,0.0,0.0,1.0,0.0,1.0,32.0,4.0,0.0,1.0,1.0,0.0,94739.2]        |0     |
|[596.0,1.0,0.0,0.0,1.0,0.0,39.0,9.0,0.0,1.0,1.0,0.0,48963.59]  

In [14]:
# Make predictions.
predictions = dtModel.transform(cust_assembled)

# Select example rows to display.
predictions.select("prediction", "Exited").show(20)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="Exited", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

+----------+------+
|prediction|Exited|
+----------+------+
|       0.0|     0|
|       0.0|     1|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       1.0|     1|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
|       0.0|     1|
|       1.0|     1|
|       0.0|     0|
|       0.0|     0|
|       1.0|     1|
|       1.0|     1|
|       0.0|     0|
|       0.0|     0|
|       0.0|     0|
+----------+------+
only showing top 20 rows

Test Error = 0.098875 


In [15]:
predictions.select("Exited","prediction").show()

+------+----------+
|Exited|prediction|
+------+----------+
|     0|       0.0|
|     1|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     1|       1.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     1|       0.0|
|     1|       1.0|
|     0|       0.0|
|     0|       0.0|
|     1|       1.0|
|     1|       1.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
+------+----------+
only showing top 20 rows



## Print Your result as the following type

In [16]:
df_private_evl.select('CustomerId','Exited').show(5)

+----------+------+
|CustomerId|Exited|
+----------+------+
|  15565701|     0|
|  15565706|     1|
|  15565796|     0|
|  15565806|     0|
|  15565878|     0|
+----------+------+
only showing top 5 rows



## Calculate f-1 score

In [17]:
from sklearn import metrics
import numpy as np
data_array =  np.array(df_private_evl.select('Exited').collect())
data_array2 = np.array(predictions.select('prediction').collect())

metrics.f1_score(data_array,data_array2)

0.7367720465890183

# ------------------------------------------------------------------------------------------------------
## Private Evaluation Part (TA will use this block to test your model)
Please 

In [18]:
df_private = spark.read.csv('private.csv',header=True,inferSchema=True)  # TA takes private dataset as example

AnalysisException: ignored

## Do prediction with your PySpark model here

In [ ]:
cust_df_private = df_private.select('CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited')

In [ ]:
# Because the sample uses an algorithm that works only with numeric features, convert them so they can be consumed
sI1 = StringIndexer(inputCol="Geography", outputCol="GeographyIndex")
en1 = OneHotEncoder(dropLast=False, inputCol="GeographyIndex", outputCol="GeographyVec")
sI2 = StringIndexer(inputCol="Gender", outputCol="GenderIndex")
en2 = OneHotEncoder(dropLast=False, inputCol="GenderIndex", outputCol="GenderVec")

# Create a new DataFrame that has had the encodings applied
encoded_final_df_private = Pipeline(stages=[sI1, en1, sI2, en2]).fit(cust_df_private).transform(cust_df_private)

In [ ]:
dtModel.load(logRegDirfilename)

In [ ]:
assembler = VectorAssembler(inputCols=[
   'CreditScore', 'GeographyVec', 'GenderVec', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'
], outputCol='features')

# Consolidate predictor columns
cust_assembled = assembler.transform(encoded_final_df_private)

# Check the resulting column
cust_assembled.select('features', 'Exited').show(20, truncate=False)

In [ ]:
# Make predictions.
predictions = dtModel.transform(cust_assembled)

# Select example rows to display.
predictions.select("prediction", "Exited").show(20)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="Exited", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

## TA will use the following function to get your prediction result (f-1 score)

In [ ]:
from sklearn import metrics
import numpy as np
data_array =  np.array(df_private.select('Exited').collect())
data_array2 = np.array(predictions.select('prediction').collect())

metrics.f1_score(data_array,data_array2)  